In [84]:
import pathlib
import cv2
import numpy as np
import time
import itertools

from collections import Counter

from tqdm import tqdm

from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from skimage import data, img_as_float
from skimage.measure import compare_ssim as ssim

import os
import pandas as pd
    
def read_and_normalize(fp):
    scaling = 0.1
    img = cv2.imread(fp, flags=cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, None,
                                 fx=scaling, fy=scaling, 
                                 interpolation=cv2.INTER_AREA)
    img = cv2.GaussianBlur(img,(7,7),0)
    img = cv2.normalize(img, None, alpha=0, beta=255, 
                                norm_type=cv2.NORM_MINMAX)
    return img

def calculate_pose_frame(file_path, window_size=5):
    """
    file_path: path to folder for one night's sleep images
    window_size: number of minutes in held position to be considered one pose
    """
    img_arr = None
    for current_file in sorted(pathlib.Path(file_path).iterdir()):
        # Need to handle case when image file name ._005.jpg appears
        # Found in .../photos_processed/2019-02-27
        if current_file.name[0] == '.':
            continue

        file_string = str(current_file)
        img = read_and_normalize(file_string)

        # Have not set up averaging window yet
        if not np.any(img_arr):
            img_shape = np.shape(img)
            img_arr = np.zeros([img_shape[0], img_shape[1], window_size])

        img = np.expand_dims(img, axis=2)

        # Still initializing window
        if not np.any(img_arr[:,:,-1]):
            img_arr = np.concatenate([img, img_arr], axis=2)
            img_arr = img_arr[:,:,0:window_size]
            continue
            
        diff_arr = img_arr - img
        diff_arr = np.abs(diff_arr)
        diff_arr = np.sum(diff_arr, axis=(0,1))
        mean_old = np.mean(diff_arr)
        std_old = np.std(diff_arr)

        print(diff_arr)
        # Normalize
        diff_arr = (diff_arr-np.min(diff_arr))/(np.max(diff_arr)-np.min(diff_arr))
        mean = np.mean(diff_arr)
        std = np.std(diff_arr)
        
        print("OLD: {:.2f}\t{:.2f}\nNEW: {:.2f}\t{:.2f}".format(std_old,mean_old,std,mean))
            
        # Insert img into arr; remove oldest img
        img_arr = np.concatenate([img, img_arr], axis=2)
        img_arr = img_arr[:,:,0:window_size]

In [85]:
dirpath = '/Volumes/NO NAME/sleeper/photos_processed/'
p = pathlib.Path(dirpath)
folders_list = ([x for x in p.iterdir() if x.is_dir()])

record = pd.DataFrame(columns=['folder_name', 'duration',
                              'count_t_frames', 't_frames_list',
                               'count_std_frames', 'std_frames_list'])

calculate_pose_frame(folders_list[4], 5)

[ 2978.  2754.  3883.  5363. 34863.]
OLD: 12481.16	9968.20
NEW: 0.39	0.22
[2431. 2227. 2287. 4100. 4690.]
OLD: 1038.05	3147.00
NEW: 0.42	0.37
[1778. 2555. 2301. 2691. 3340.]
OLD: 510.01	2533.00
NEW: 0.33	0.48
[1893. 1703. 2560. 3012. 3090.]
OLD: 566.71	2451.60
NEW: 0.41	0.54
[1957. 2398. 2910. 3105. 3981.]
OLD: 685.20	2870.20
NEW: 0.34	0.45
[6029. 6346. 6169. 6665. 6944.]
OLD: 333.29	6430.60
NEW: 0.36	0.44
[1517. 6302. 6857. 6660. 7284.]
OLD: 2127.27	5724.00
NEW: 0.37	0.73
[6004. 5851. 1962. 2905. 2954.]
OLD: 1665.40	3935.20
NEW: 0.41	0.49
[1481. 6131. 5986. 2211. 2918.]
OLD: 1943.08	3745.40
NEW: 0.42	0.49
[6308. 6257. 1937. 2658. 6793.]
OLD: 2056.86	4790.60
NEW: 0.42	0.59
[2837. 6909. 6676. 3156. 3479.]
OLD: 1793.91	4611.40
NEW: 0.44	0.44
[2787. 2172. 6040. 5971. 2009.]
OLD: 1822.91	3795.80
NEW: 0.45	0.44
[2404. 2907. 1622. 6682. 6501.]
OLD: 2137.39	4023.20
NEW: 0.42	0.47
[1522. 2272. 2585. 1810. 6580.]
OLD: 1849.78	2953.80
NEW: 0.37	0.28
[1697. 2047. 2007. 2378. 2197.]
OLD: 225.61	20

[1528. 1690. 1538. 1802. 1648.]
OLD: 101.73	1641.20
NEW: 0.37	0.41
[1487. 1731. 1779. 1743. 1823.]
OLD: 117.26	1712.60
NEW: 0.35	0.67
[13370. 13537. 13553. 13521. 13115.]
OLD: 165.59	13419.20
NEW: 0.38	0.69
[7702. 7190. 7125. 7181. 7211.]
OLD: 212.02	7281.80
NEW: 0.37	0.27
[1660. 8544. 6780. 6701. 6835.]
OLD: 2325.96	6104.00
NEW: 0.34	0.65
[1601. 1643. 7885. 6973. 7084.]
OLD: 2806.23	5037.20
NEW: 0.45	0.55
[1607. 1374. 1796. 8580. 6728.]
OLD: 3029.75	4017.00
NEW: 0.42	0.37
[ 6684.  6979.  6752.  7268. 13614.]
OLD: 2685.08	8259.40
NEW: 0.39	0.23
[1588. 6628. 7181. 6708. 7272.]
OLD: 2158.54	5875.40
NEW: 0.38	0.75
[1509. 1399. 6805. 7200. 6863.]
OLD: 2699.02	4755.20
NEW: 0.47	0.58
[7322. 7337. 7111. 1845. 1460.]
OLD: 2749.33	5015.00
NEW: 0.47	0.60
[1558. 7058. 7081. 6951. 1717.]
OLD: 2642.62	4873.00
NEW: 0.48	0.60
[7083. 7395. 1635. 1928. 1634.]
OLD: 2701.63	3935.00
NEW: 0.47	0.40
[6875. 1558. 1894. 6922. 6827.]
OLD: 2524.74	4815.20
NEW: 0.47	0.61
[1912. 7467. 1792. 1806. 7498.]
OLD: 2766

[ 1202.  1142.  1429.  1611. 13930.]
OLD: 5036.37	3862.80
NEW: 0.39	0.21
[1661. 1587. 1433. 1468. 1308.]
OLD: 122.88	1491.40
NEW: 0.35	0.52
[1455. 1894. 1818. 1794. 1681.]
OLD: 152.81	1728.40
NEW: 0.35	0.62
[1514. 1421. 1760. 1844. 1732.]
OLD: 159.57	1654.20
NEW: 0.38	0.55
[1619. 1919. 1774. 2075. 2127.]
OLD: 188.32	1902.80
NEW: 0.37	0.56
[1519. 1462. 1674. 1613. 1538.]
OLD: 74.25	1561.20
NEW: 0.35	0.47
[711685. 711434. 711495. 711571. 711596.]
OLD: 86.11	711556.20
NEW: 0.34	0.49
[ 69396. 766925. 766598. 766711. 766821.]
OLD: 278947.12	627290.20
NEW: 0.40	0.80
[  5060.  72868. 771077. 770750. 770863.]
OLD: 359212.90	478123.60
NEW: 0.47	0.62
[  3764.   7992.  75444. 774237. 773912.]
OLD: 365863.17	327069.80
NEW: 0.47	0.42
[  3535.   5289.   9443.  77459. 775062.]
OLD: 301727.13	174157.60
NEW: 0.39	0.22
[ 4450.  5655.  7783. 11933. 79161.]
OLD: 28795.14	21796.40
NEW: 0.39	0.23
[167247. 168007. 169322. 169470. 170782.]
OLD: 1229.08	168965.60
NEW: 0.35	0.49
[  7944. 169181. 170013. 171402.

In [ ]:
for f in folders_list:
    t_frames_list, std_frames_list = pr.get_sift_frames(f, verbose=0)

    folders_list = sorted(pathlib.Path(f).iterdir())
    first_file_int = get_index_of_file(f, 0)
    last_file_int = get_index_of_file(f, -1)
    duration = last_file_int - first_file_int
    
    record = record.append({'folder_name': f.stem,
                            'duration': duration,
                            'count_t_frames': len(t_frames_list),
                            't_frames_list': t_frames_list,
                            'count_std_frames': len(std_frames_list),
                            'std_frames_list': std_frames_list},
                           ignore_index=True)

display(record)
record.to_csv('/Volumes/NO NAME/sleeper/record.csv')

In [13]:
def get_index_of_file(file_path, index):
    list_of_files = sorted(pathlib.Path(file_path).iterdir())
    f = list_of_files[index]
    try:
        f = int(f.stem)
    except ValueError:
        for current_file in list_of_files:
            if current_file.name[0] != '.':
                f = current_file
                break
        f = int(f.stem)
    return f

def get_files_list(file_path, transition_frames):
    files_list = list()
    start_int = get_index_of_file(file_path, 0)
    for i in transition_frames:
        files_list.append(str(pathlib.Path(file_path)
                      .joinpath('{:03d}.jpg'
                                .format(i+start_int))))
    return files_list


def display_images(self, files_list):
    fig = plt.figure(1, (14., 10.))
    img_per_row = 7
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(len(files_list)//img_per_row+1, img_per_row),  # creates 2x2 grid of axes
                 axes_pad=0.05,  # pad between axes in inch.
                 )
    #for i,f in tqdm(enumerate(files_list)):
    for i,f in enumerate(files_list):
        img = cv2.imread(f, flags=cv2.IMREAD_GRAYSCALE)
        if self.equalize:
            img = cv2.equalizeHist(img)
        img = img_as_float(img)
        grid[i].axis('off')
        grid[i].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    return grid


def run(self, file_path, verbose=0):
    t_frames_list = list()
    std_frames_list = list()
    for scaling in scaling_options:
        (transition_frames, transition_values, 
         avg, std,
         transition_frames_std, transition_values_std) = self.calculate_t_frames(file_path, scaling)
        files_list = self.get_files_list(file_path, transition_frames)
        std_files_list = self.get_files_list(file_path, transition_frames_std)

        t_frames_list.append(transition_frames)
        std_frames_list.append(transition_frames_std)
        if verbose > 0:
            self.display_images(files_list)
            plt.show()
            print("Count: {}\n".format(len(transition_frames)))
            time.sleep(1)
            self.display_images(std_files_list)
            plt.show()
            print("Count: {}\n".format(len(transition_frames_std)))
    if verbose > 1:
        print("T Frames:\n{} \n\nSTD Frames:\n{}".format(t_frames_list, std_frames_list))

    return (t_frames_list, std_frames_list)

def get_counts(self, frames_list):
    counts = Counter()
    for t_frames in frames_list:
        counts += Counter(t_frames)
    c = dict(counts)
    d = [k for k,v in c.items() if v > 4]
    return (d)